In [8]:
import pandas as pd
import os
import yfinance as yf
from yahooquery import Ticker
from yahooquery import Screener
from datetime import datetime
s = Screener()

data_path = r'C:\Users\pythonProject\data\data_produced\yquerry_screens\options'
today = datetime.today().strftime('%Y%m%d')
file_name = data_path + f"\\{today}_option_analysis_2tickers.csv"
#excel_file_path = os.path.join(data_path, scr_file_name) 


In [51]:
stocklist = ['FSR', 'RIVN']
fsr_future_price = 10
rivn_future_price = 30

In [42]:
t = Ticker(stocklist, asynchronous=True)
option_df = t.option_chain


In [25]:
"""
next , fetch latest prices for the stocks in the screen data list

"""

def fetch_latest_prices(tickers):
    error_list = []
    stock_data = pd.DataFrame()  # Initialize an empty DataFrame

    for ticker in tickers:
        try:
            data = yf.download(ticker, period="1d", interval="1m")
            latest_price = data["Close"].iloc[-1]
            stock_data = pd.concat([stock_data, pd.DataFrame({"Ticker": ticker, "Latest_Price": latest_price}, index=[0])])
        except Exception as e:
            print(f"Error fetching price for {ticker}: {e}")
            error_list.append(ticker)
    
    stock_data = stock_data.sort_values(by="Ticker")
    return stock_data, error_list

# Example usage:
#tickers = ["AAPL", "GOOG", "MSFT", "INVALID"]  # Replace with your list of tickers
stock_data, error_list = fetch_latest_prices(stocklist)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [44]:
option_df = option_df.reset_index(level=['optionType'], drop=False).reset_index(level=['expiration'], drop=False)


,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney
symbol,,,,,,,,,,,,,,,,
FSR,2024-01-26,calls,FSR240126C00000500,0.5,USD,0.24,-0.09,-27.272732,44.0,234,0.22,0.25,REGULAR,2024-01-25 15:41:00,3.750001,True
FSR,2024-01-26,calls,FSR240126C00001000,1.0,USD,0.01,-0.01,-50.000000,629.0,19928,0.00,0.01,REGULAR,2024-01-25 15:44:42,2.750003,False


In [45]:
stock_data.set_index('Ticker', inplace=True)

In [66]:
temp_df = option_df.copy()
import numpy as np 
temp_df["break_even"] = temp_df["strike"] + temp_df["lastPrice"]
temp_df["futurePrice"] = np.where(temp_df.index == "FSR",
                                    fsr_future_price - temp_df["strike"],
                                    rivn_future_price - temp_df["strike"])
temp_df["return"] = round((temp_df["futurePrice"] - temp_df["lastPrice"]) / temp_df["lastPrice"] , 2)
temp_df = temp_df.sort_values(by = "return")
temp_df.to_csv(file_name)
temp_df

,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney,break_even,futurePrice,return
symbol,,,,,,,,,,,,,,,,,,,
RIVN,2024-03-15,calls,RIVN240315C00040000,40.0,USD,0.01,-0.02,-66.666670,36.0,9606,0.01,0.03,REGULAR,2024-01-25 15:32:45,1.078130,False,40.01,-10.0,-1001.0
RIVN,2024-01-26,calls,RIVN240126C00035000,35.0,USD,0.01,0.00,0.000000,1.0,892,0.00,0.02,REGULAR,2024-01-22 14:36:58,4.500004,False,35.01,-5.0,-501.0
RIVN,2024-02-09,calls,RIVN240209C00035000,35.0,USD,0.01,0.00,0.000000,0.0,35,0.00,0.03,REGULAR,2024-01-04 14:36:00,1.656252,False,35.01,-5.0,-501.0
RIVN,2024-02-16,calls,RIVN240216C00035000,35.0,USD,0.01,0.00,0.000000,1.0,919,0.01,0.02,REGULAR,2024-01-24 17:22:30,1.375003,False,35.01,-5.0,-501.0
RIVN,2025-01-17,calls,RIVN250117C00080000,80.0,USD,0.10,-0.02,-16.666664,23.0,12962,0.11,0.13,REGULAR,2024-01-25 14:50:20,0.775393,False,80.10,-50.0,-501.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RIVN,2024-01-26,puts,RIVN240126P00011000,11.0,USD,0.01,0.00,0.000000,87.0,98,0.00,0.00,REGULAR,2024-01-19 20:30:14,0.500005,False,11.01,19.0,1899.0
RIVN,2024-01-26,puts,RIVN240126P00010000,10.0,USD,0.01,0.00,0.000000,87.0,156,0.00,0.01,REGULAR,2024-01-19 18:46:19,2.125005,False,10.01,20.0,1999.0
RIVN,2024-02-02,puts,RIVN240202P00010000,10.0,USD,0.01,0.00,0.000000,0.0,10,0.00,0.06,REGULAR,2024-01-22 15:44:22,1.312503,False,10.01,20.0,1999.0
